In [26]:
html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Air Alarm Forecast</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            padding: 2em;
            background: #f9f9f9;
        }

        h1 {
            color: #333;
        }

        form {
            margin-bottom: 1.5em;
        }

        select, input[type="date"], input[type="time"] {
            padding: 0.5em;
            font-size: 1em;
            margin-bottom: 1em;
        }

        button {
            padding: 0.5em 1em;
            font-size: 1em;
            background: #007bff;
            color: white;
            border: none;
            border-radius: 4px;
        }

        .forecast-container {
            margin-top: 2em;
        }

        .region-name {
            font-size: 1.4em;
            margin-top: 1em;
            font-weight: bold;
        }

        .hour {
            display: inline-block;
            margin: 0.25em 0.5em;
            font-size: 0.95em;
        }

        .dot {
            height: 12px;
            width: 12px;
            border-radius: 50%;
            display: inline-block;
            margin-left: 6px;
        }

        .dot-green {
            background-color: #28a745;
        }

        .dot-red {
            background-color: #dc3545;
        }

        #previous-prediction-fields {
            display: none;
        }
        
        .day-title {
            font-size: 0.9em;
            font-weight: bold;
            margin: 30px 0 10px;
        }
        
        .forecast-table {
            border-collapse: collapse;
            width: 100%;
            margin-bottom: 40px;
        }
        
        .forecast-table th,
        .forecast-table td {
            text-align: center;
            padding: 8px;
        }
        
        .forecast-table th {
            background-color: #f2f2f2;
            font-weight: bold;
        }
        
        .forecast-table tbody tr:nth-child(even) {
            background-color: #f9f9f9;
        }
        
        .forecast-table tbody tr:nth-child(odd) {
            background-color: #ffffff;
        }
        
        .status.safe {
            color: green;
        }
        
        .status.alarm {
            color: red;
        }
        
        .region-name {
            text-align: left;
            font-weight: bold;
            padding-top: 12px;
            font-size: 18px;
        }

    </style>

    <script>
        document.addEventListener('DOMContentLoaded', function () {
            const form = document.getElementById('forecast-form');

            form.addEventListener('submit', function(event) {
                console.log("Form submitted");

                const mode = document.querySelector('input[name="mode"]:checked').value;
                const location = document.getElementById('location').value;

                if (!location) {
                    alert('Please select a region.');
                    event.preventDefault();
                    return;
                }

                if (mode === 'previous') {
                    const date = document.querySelector('input[name="date"]').value;
                    const time = document.querySelector('input[name="time"]').value;

                    if (!date || !time) {
                        alert('Please select both date and time for previous prediction.');
                        event.preventDefault();
                        return;
                    }
                }

                console.log("Form is ready to submit.");
            });

            document.querySelectorAll('input[name="mode"]').forEach(function(radio) {
                radio.addEventListener('change', function() {
                    const previousFields = document.getElementById('previous-prediction-fields');
                    previousFields.style.display = this.value === 'previous' ? 'block' : 'none';
                });
            });
        });
    </script>
</head>
<body>
    <h1>Air Alarm Forecast</h1>

    <form method="POST" action="/get_forecast" id="forecast-form">
        <label for="location">Choose a region:</label>
        <select name="location" id="location" required>
            <option value="">-- Select Region --</option>
            <option value="all">All Regions</option>
            {% set region_map = {
                1: "Simferopol",
                2: "Vinnytsia",
                3: "Lutsk",
                4: "Dnipro",
                5: "Donetsk",
                6: "Zhytomyr",
                7: "Uzhgorod",
                8: "Zaporizhzhia",
                9: "Ivano-Frankivsk",
                10: "Kyiv_obl",
                11: "Kropyvnytskyi",
                12: "Luhansk",
                13: "Lviv",
                14: "Mykolaiv",
                15: "Odesa",
                16: "Poltava",
                17: "Rivne",
                18: "Sumy",
                19: "Ternopil",
                20: "Kharkiv",
                21: "Kherson",
                22: "Khmelnytskyi",
                23: "Cherkasy",
                24: "Chernivtsi",
                25: "Chernihiv",
                26: "Kyiv"
            } %}
            {% for id, name in region_map.items() %}
                <option value="{{ id }}">{{ name }}</option>
            {% endfor %}
        </select>

        <label for="mode">Choose prediction type:</label><br>
        <input type="radio" name="mode" value="new" checked> New Prediction<br>
        <input type="radio" name="mode" value="previous"> Previous Prediction<br>

        <div id="previous-prediction-fields">
            <label for="date">Select Date (up to today):</label>
            <input type="date" name="date" max="{{ now }}"><br>

            <label for="time">Select Time:</label>
            <input type="time" name="time"><br>
        </div>

        <button type="submit" id="submit-btn">Get Forecast</button>
    </form>
    
    {% if forecast %}
        {% for region_name, days in forecast.items() %}
            <div class="region-name">{{ region_name }}</div>
    
            {% for day_name in ['today', 'tomorrow'] %}
                <div class="day-title">{{ day_name|capitalize }}</div>
                <table class="forecast-table">
                    <thead>
                        <tr>
                            <th>Hour</th>
                            {% for hour in days[day_name].keys() %}
                                <th>{{ hour }}</th>
                            {% endfor %}
                        </tr>
                    </thead>
                    <tbody>
                        <tr>
                            <td>Status</td>
                            {% for hour, value in days[day_name].items() %}
                                <td>
                                    <span class="status {% if value == 1 %}alarm{% else %}safe{% endif %}">
                                        {% if value == 1 %}⚠️<br>Alarm!{% else %}✅<br>Safety{% endif %}
                                    </span>
                                </td>
                            {% endfor %}
                        </tr>
                    </tbody>
                </table>
            {% endfor %}
        {% endfor %}
    {% endif %}


    {% if error %}
        <p style="color: red;">Error: {{ error }}</p>
    {% endif %}
</body>
</html>

"""

In [27]:
with open("index.html", "w") as file:
    file.write(html_code)